# TSA Chapter 5: Bitcoin GARCH Model

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/QuantLet/TSA/blob/main/TSA_ch5/TSA_ch5_btc_garch/TSA_ch5_btc_garch.ipynb)

Fitting GARCH(1,1) model to Bitcoin returns.

In [ ]:
!pip install numpy pandas matplotlib scipy yfinance arch statsmodels -q

In [ ]:
import warnings
warnings.filterwarnings('ignore')

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
from scipy import stats
# Chart style
BLUE = '#1A3A6E'; RED = '#DC3545'; GREEN = '#2E7D32'
ORANGE = '#E67E22'; GRAY = '#666666'; PURPLE = '#8E44AD'

plt.rcParams.update({
    'figure.facecolor': 'none', 'axes.facecolor': 'none',
    'savefig.facecolor': 'none', 'savefig.transparent': True,
    'axes.grid': False, 'font.size': 10, 'axes.labelsize': 11,
    'axes.titlesize': 12, 'legend.fontsize': 9, 'xtick.labelsize': 9,
    'ytick.labelsize': 9, 'axes.spines.top': False, 'axes.spines.right': False,
    'lines.linewidth': 1.0, 'axes.linewidth': 0.6,
    'legend.facecolor': 'none', 'legend.framealpha': 0, 'legend.edgecolor': 'none',
    'figure.dpi': 150,
})

def save_chart(fig, name):
    fig.savefig(f'{name}.pdf', bbox_inches='tight', transparent=True, dpi=150)
    fig.savefig(f'{name}.png', bbox_inches='tight', transparent=True, dpi=150)
    print(f'Saved: {name}')

def legend_bottom(ax, ncol=2, y=-0.18):
    ax.legend(loc='upper center', bbox_to_anchor=(0.5, y), ncol=ncol, frameon=False)
import yfinance as yf
from arch import arch_model

In [ ]:
import yfinance as yf

# Download Bitcoin data
btc = yf.download('BTC-USD', start='2019-01-01', end='2025-12-31', progress=False)
if isinstance(btc.columns, pd.MultiIndex):
    btc.columns = btc.columns.get_level_values(0)
btc['Return'] = btc['Close'].pct_change() * 100
btc.dropna(inplace=True)
btc_returns = btc['Return']
print(f"Bitcoin: {len(btc)} observations")

In [ ]:
# Fit GARCH(1,1) to Bitcoin
am = arch_model(btc_returns, vol='Garch', p=1, q=1, mean='Constant', dist='t')
res = am.fit(disp='off')
print(res.summary())

In [ ]:
# Plot conditional volatility
fig, ax = plt.subplots(figsize=(10, 4))
ax.plot(btc.index, res.conditional_volatility, color=RED, lw=0.6, label='GARCH(1,1) $\\sigma_t$')
ax.fill_between(btc.index, 0, res.conditional_volatility, color=RED, alpha=0.2)
ax.axhline(res.conditional_volatility.mean(), color=GREEN, ls='--', lw=1,
           label=f'Mean vol = {res.conditional_volatility.mean():.2f}%')
ax.set_title('Bitcoin GARCH(1,1) Conditional Volatility', fontweight='bold')
ax.set_ylabel('Volatility (%)'); ax.set_xlabel('Date')
ax.xaxis.set_major_locator(mdates.YearLocator())
ax.xaxis.set_major_formatter(mdates.DateFormatter('%Y'))
legend_bottom(ax, ncol=2, y=-0.18)
fig.tight_layout(); fig.subplots_adjust(bottom=0.20)
plt.show()